In [109]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

Let's begin by reading the expression matrix:

In [2]:
expression_matrix = pd.read_csv("../R_code/exp_matrix_cancer.csv")
expression_matrix.drop(['Unnamed: 0'], axis=1, inplace=True)
expression_matrix.head(5)

,1007_s_at,1053_at,117_at,121_at,1255_g_at,1294_at,1316_at,1320_at,1405_i_at,1431_at,...,AFFX-r2-P1-cre-3_at,AFFX-r2-P1-cre-5_at,AFFX-ThrX-3_at,AFFX-ThrX-5_at,AFFX-ThrX-M_at,AFFX-TrpnX-3_at,AFFX-TrpnX-5_at,AFFX-TrpnX-M_at,sample_names,has_cancer
0,8.496339,5.944231,5.838808,7.180569,2.759084,7.383376,7.096457,4.803199,6.957270,3.321985,...,13.985169,13.774519,9.948910,6.152068,8.171463,2.693050,3.063041,3.117905,GSM1045191_N1_15_12_04.CEL,0
1,7.354706,5.633305,6.325852,7.901653,4.077464,7.294313,6.295099,4.436363,6.889800,3.972205,...,14.241674,14.060209,13.450387,10.559453,12.546760,2.702545,3.845348,3.677584,GSM1045192_N4_14_12_04.CEL,0
2,6.898585,5.871544,5.514367,7.718790,3.175172,8.083949,5.853364,4.366101,5.378046,3.347600,...,14.205027,14.096933,13.243120,10.395400,12.455792,2.722887,3.705345,3.173042,GSM1045193_N5_15_12_04.CEL,0
3,9.640469,5.893211,5.668953,7.640794,3.067836,7.310535,6.694541,4.268436,5.611767,3.496480,...,14.348157,14.244256,11.973984,7.766641,10.270549,2.867993,3.215214,3.420670,GSM1045194_N6_14_12_04.CEL,0
4,8.175582,6.151961,6.065677,7.563042,2.980283,7.557248,7.165543,4.727182,5.927592,3.372731,...,14.126333,13.988384,10.456795,6.946789,8.623066,2.770782,3.170171,3.164002,GSM1045195_P1_15_12_04.CEL,0


And then also the annotation matrix:

In [3]:
annotation = pd.read_csv("../R_code/annotation.csv", encoding='utf-8')
annotation.head(5)

,ensembl_gene_id,affy_hg_u133_plus_2,hgnc_symbol,description,refseq_mrna,refseq_ncrna
0,ENSG00000198888,1553551_s_at,MT-ND1,mitochondrially encoded NADH:ubiquinone oxidor...,NaN,NaN
1,ENSG00000210100,1553551_s_at,MT-TI,mitochondrially encoded tRNA-Ile (AUU/C) [Sour...,NaN,NaN
2,ENSG00000210112,1553551_s_at,MT-TM,mitochondrially encoded tRNA-Met (AUA/G) [Sour...,NaN,NaN
3,ENSG00000198763,1553551_s_at,MT-ND2,mitochondrially encoded NADH:ubiquinone oxidor...,NaN,NaN
4,ENSG00000198804,1553538_s_at,MT-CO1,mitochondrially encoded cytochrome c oxidase I...,NaN,NaN


We filter the expression matrix with only the genes of interest:

In [51]:
genes_of_interest = ["SNORA1", "SNORA12", "SNORA14B", "SNORA16A", "SNORA21", "SNORA23", "SNORA24", "SNORA32", "SNORA38", "SNORA44", "SNORA48", "SNORA49", "SNORA52", "SNORA53", "SNORA57", "SNORA61", "SNORA63", "SNORA64", "SNORA65", "SNORA70", "SNORA71C", "SNORA73A", "SNORA73B", "SNORA75", "SNORA78", "SNORA8", "SNORD10", "SNORD102", "SNORD104", "SNORD105", "SNORD105B", "SNORD108", "SNORD110", "SNORD111B", "SNORD113-3", "SNORD113-4", "SNORD114-1", "SNORD114-13", "SNORD114-14", "SNORD114-19", "SNORD114-20", "SNORD114-21", "SNORD115-23", "SNORD115-32", "SNORD116-13", "SNORD119", "SNORD12", "SNORD12B", "SNORD13", "SNORD14A", "SNORD14C", "SNORD14D", "SNORD15B", "SNORD16", "SNORD17", "SNORD18A", "SNORD1B", "SNORD20", "SNORD22", "SNORD26", "SNORD28", "SNORD29", "SNORD32A", "SNORD33", "SNORD34", "SNORD35A", "SNORD36A", "SNORD36B", "SNORD38A", "SNORD3A", "SNORD3D", "SNORD41", "SNORD42A", "SNORD42B", "SNORD44", "SNORD45A", "SNORD47", "SNORD49B", "SNORD4B", "SNORD5", "SNORD50A",
                     "SNORD52", "SNORD53", "SNORD54", "SNORD55", "SNORD56", "SNORD57", "SNORD58A", "SNORD58C", "SNORD59A", "SNORD60", "SNORD63", "SNORD64", "SNORD65", "SNORD68", "SNORD69", "SNORD71", "SNORD74", "SNORD76", "SNORD8", "SNORD81", "SNORD82", "SNORD84", "SNORD86", "SNORD87", "SNORD89", "SNORD94", "SNORD96A", "SNORD97", "SNORD99", "TMX1", "ZFAS1", "CDKN2B-AS1", "CWF19L1", "EIF4A1", "EP400", "HIF1A-AS2", "MEG8", "NOP56", "RACK1", "RPL13A", "RPS13", "SCARNA12", "SNHG12", "SNHG20", "SNHG5", "SNHG6", "SNHG7", "SNHG8", "AP1G1", "CFDP1", "CHD8", "DDX39B", "EEF2", "EIF4A2", "EIF4G2", "GAS5", "GNL3", "HSPA8", "HSPA9", "IPO7", "MYRIP", "NAN", "NCL", "NFATC3", "PCAT4", "PPAN", "PRKAA1", "PRRC2A", "PTCD3", "RABGGTB", "RNF149", "RPL10", "RPL12", "RPL13", "RPL17", "RPL21", "RPL23", "RPL23A", "RPL4", "RPL7A", "RPLP2", "RPS11", "RPS2", "RPS20", "RPS3", "RPS8", "SF3B3", "SLC25A3", "SNHG1", "SNHG3", "SNHG9", "SNORD1C", "SNORD35B", "SNORD37", "SNRPB", "SNX5", "TAF1D", "TNPO2", "TOMM20", "WDR43"]
ids_of_interest = annotation[annotation['hgnc_symbol'].isin(genes_of_interest)]
ids_of_interest = ids_of_interest['affy_hg_u133_plus_2'].unique()

expression_matrix_filtered = expression_matrix[ids_of_interest]

In [110]:
expression_matrix_filtered["has_cancer"] = expression_matrix["has_cancer"]
expression_matrix_filtered.head(5)

,1555996_s_at,1554591_at,1558619_at,1552729_at,1555177_at,1558954_at,1557964_at,1566340_at,1560023_x_at,1560021_at,...,237945_at,236588_at,239266_at,235536_at,242856_at,241840_at,244669_at,241448_at,241892_at,has_cancer
0,2.751045,3.780635,5.561609,4.797828,3.319152,5.552243,6.202000,4.548862,3.872257,3.230122,...,4.878438,5.172884,4.674907,5.859758,4.328895,3.948260,9.119880,5.154011,3.863078,0
1,3.859755,5.200241,6.102399,4.359311,3.279701,6.260842,6.592251,5.067787,4.560927,3.673857,...,5.060370,5.653306,5.973646,5.846220,4.087314,3.971016,7.127229,4.562605,4.711682,0
2,3.535307,5.732324,5.212332,5.096507,3.623445,6.460197,6.504574,5.310705,3.678081,2.924379,...,4.710243,6.368515,5.904419,4.712503,5.199880,4.186570,8.256072,6.618734,4.951906,0
3,3.168197,5.126305,5.821668,5.990126,3.355111,6.429544,6.928331,5.772909,4.247690,3.309662,...,4.071639,5.214243,6.061341,5.640685,4.806696,4.619876,8.479675,4.893531,4.098783,0
4,2.821168,4.250059,5.973408,4.619057,2.759373,5.834466,6.299788,4.677527,3.657642,3.143385,...,4.471927,5.175007,4.168242,5.891731,4.455629,4.295718,8.393797,4.568402,3.650335,0


In [19]:
sample_names = expression_matrix['sample_names']
#expression_matrix.drop(['sample_names'], axis=1, inplace=True)

Güt. Now we can start doing some ML. First, we fit a Multinomial Naive Bayes:

In [73]:
X_train, X_test, Y_train, Y_test = train_test_split(
    expression_matrix_filtered.drop(['has_cancer'], axis=1), expression_matrix_filtered['has_cancer'], test_size=0.2, random_state=226506)

tuned_nb = GridSearchCV(MultinomialNB(), {}, refit=True, cv=10, verbose=4)
tuned_nb.fit(X_train, Y_train)

preds_mnb = tuned_nb.predict(X_test)

preds_mnb = preds_mnb.astype(int)
Y_test = Y_test.astype(int)
print(f"Accuracy: {metrics.accuracy_score(Y_test, preds_mnb)}")
print(f"Precision: {metrics.precision_score(Y_test, preds_mnb)}")
print(f"Recall: {metrics.recall_score(Y_test, preds_mnb)}")
print(f"F1: {metrics.f1_score(Y_test, preds_mnb)}")
print(metrics.classification_report(Y_test, preds_mnb))


Fitting 10 folds for each of 1 candidates, totalling 10 fits
[CV 1/10] END .................................., score=0.800 total time=   0.0s
[CV 2/10] END .................................., score=0.960 total time=   0.0s
[CV 3/10] END .................................., score=0.840 total time=   0.0s
[CV 4/10] END .................................., score=0.920 total time=   0.0s
[CV 5/10] END .................................., score=0.880 total time=   0.0s
[CV 6/10] END .................................., score=0.958 total time=   0.0s
[CV 7/10] END .................................., score=1.000 total time=   0.0s
[CV 8/10] END .................................., score=0.875 total time=   0.0s
[CV 9/10] END .................................., score=0.875 total time=   0.0s
[CV 10/10] END ................................., score=0.917 total time=   0.0s
Accuracy: 0.8548387096774194
Precision: 0.8979591836734694
Recall: 0.9166666666666666
F1: 0.9072164948453607
              precis

Decent performance, but let's check if an SVM can do better:

In [68]:
tune_grid = {
    'gamma': [0.00001, 0.0001, 0.001, 0.1, 1, 'scale', 'auto'],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'degree': [1, 2, 3, 4],
    'C': [0.1, 1, 10, 100]
}

tuned_svm = GridSearchCV(SVC(), tune_grid, refit=True, cv=10, verbose=4)
tuned_svm.fit(X_train, Y_train)

preds_svm = tuned_svm.predict(X_test)

preds_svm = preds_svm.astype(int)
print(f"Accuracy: {metrics.accuracy_score(Y_test, preds_svm)}")
print(f"Precision: {metrics.precision_score(Y_test, preds_svm)}")
print(f"Recall: {metrics.recall_score(Y_test, preds_svm)}")
print(f"F1: {metrics.f1_score(Y_test, preds_svm)}")
print(metrics.classification_report(Y_test, preds_svm))


Fitting 10 folds for each of 448 candidates, totalling 4480 fits
[CV 1/10] END C=0.1, degree=1, gamma=1e-05, kernel=linear;, score=0.920 total time=   0.0s
[CV 2/10] END C=0.1, degree=1, gamma=1e-05, kernel=linear;, score=1.000 total time=   0.0s
[CV 3/10] END C=0.1, degree=1, gamma=1e-05, kernel=linear;, score=0.960 total time=   0.0s
[CV 4/10] END C=0.1, degree=1, gamma=1e-05, kernel=linear;, score=1.000 total time=   0.0s
[CV 5/10] END C=0.1, degree=1, gamma=1e-05, kernel=linear;, score=0.960 total time=   0.0s
[CV 6/10] END C=0.1, degree=1, gamma=1e-05, kernel=linear;, score=1.000 total time=   0.0s
[CV 7/10] END C=0.1, degree=1, gamma=1e-05, kernel=linear;, score=1.000 total time=   0.0s
[CV 8/10] END C=0.1, degree=1, gamma=1e-05, kernel=linear;, score=1.000 total time=   0.0s
[CV 9/10] END C=0.1, degree=1, gamma=1e-05, kernel=linear;, score=1.000 total time=   0.0s
[CV 10/10] END C=0.1, degree=1, gamma=1e-05, kernel=linear;, score=1.000 total time=   0.0s
[CV 1/10] END C=0.1, deg

Much better. We can also check what are the best parameters:

In [70]:
print(tuned_svm.best_estimator_)

SVC(C=0.1, degree=1, gamma=1e-05, kernel='linear')


Now we can try to fit a ridge regression:

In [90]:
tune_grid = {
    'C': [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 1, 10, 100],
    'penalty': ['l2']
}

tuned_ridge = GridSearchCV(LogisticRegression(max_iter=1000), tune_grid, cv=10, verbose=4)

tuned_ridge.fit(X_train, Y_train)

preds_ridge = tuned_ridge.predict(X_test)

preds_ridge = preds_ridge.astype(int)
print(f"Accuracy: {metrics.accuracy_score(Y_test, preds_ridge)}")
print(f"Precision: {metrics.precision_score(Y_test, preds_ridge)}")
print(f"Recall: {metrics.recall_score(Y_test, preds_ridge)}")
print(f"F1: {metrics.f1_score(Y_test, preds_ridge)}")
print(metrics.classification_report(Y_test, preds_ridge))

Fitting 10 folds for each of 23 candidates, totalling 230 fits
[CV 1/10] END ...............C=0.01, penalty=l2;, score=0.920 total time=   0.0s
[CV 2/10] END ...............C=0.01, penalty=l2;, score=0.920 total time=   0.0s
[CV 3/10] END ...............C=0.01, penalty=l2;, score=0.960 total time=   0.0s
[CV 4/10] END ...............C=0.01, penalty=l2;, score=0.960 total time=   0.0s
[CV 5/10] END ...............C=0.01, penalty=l2;, score=0.960 total time=   0.0s
[CV 6/10] END ...............C=0.01, penalty=l2;, score=0.958 total time=   0.0s
[CV 7/10] END ...............C=0.01, penalty=l2;, score=1.000 total time=   0.0s
[CV 8/10] END ...............C=0.01, penalty=l2;, score=0.917 total time=   0.0s
[CV 9/10] END ...............C=0.01, penalty=l2;, score=1.000 total time=   0.0s
[CV 10/10] END ..............C=0.01, penalty=l2;, score=0.958 total time=   0.0s
[CV 1/10] END ...............C=0.02, penalty=l2;, score=0.920 total time=   0.0s
[CV 2/10] END ...............C=0.02, penalty=l

It also gives us very good results. Let's check the parameters:

In [92]:
print(tuned_ridge.best_estimator_)

LogisticRegression(C=10, max_iter=1000)


Let's also check the coefficients:

In [105]:
ridge = LogisticRegression(C=10, penalty='l2', max_iter=1000)

ridge.fit(X_train, Y_train)
print(ridge.coef_)

[[ 0.05438692  0.08170905  0.04891421 -0.49478602 -0.04299994  0.00377561
  -0.3985913   0.10359035 -0.53726929 -0.41060791  0.14682986 -0.06087096
  -0.0659303   0.15681673 -0.07257855 -0.29644671 -0.05060175 -0.00371523
   0.33665877  0.14137493  0.05607593 -0.67116759 -0.32813481 -0.32123246
  -0.15666301 -0.35089702 -0.02097143  0.19845362 -0.70320283  0.17206215
  -0.37050835 -0.1629628  -0.00849114 -0.09558741  0.01726735 -0.0280372
  -0.14580425 -0.53323225  0.48242882 -0.1909962  -0.32154261 -0.2479101
  -0.00932726  0.11493101  0.22670531  0.32042563  0.41636539  0.30316587
   0.02871651  0.09003023 -0.51075295  0.33629659  0.05476497  0.16801595
   0.80984145  0.80028437  0.17649455 -0.07957763 -0.12684077 -0.03803103
  -0.20800291  0.0028514  -0.04489775 -0.48601379 -0.03316757  0.07545592
  -0.07954453 -0.02874781  0.31192839 -0.89762969 -0.20999263  0.42145435
  -0.13523484 -0.13469599  0.19743305  0.15478522 -0.07429535 -0.06117743
  -0.32581825  1.1433579   0.05793961  0

Finally, we fit a lasso regression to see if there is a comparable difference with the ridge:

In [100]:
tune_grid = {
    'C': [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 1, 10, 100],
    'penalty': ['l1']
}

tuned_lasso = GridSearchCV(LogisticRegression(max_iter=10000, solver='saga'), tune_grid, cv=10, verbose=4)

tuned_lasso.fit(X_train, Y_train)

preds_lasso = tuned_lasso.predict(X_test)

preds_lasso = preds_lasso.astype(int)
print(f"Accuracy: {metrics.accuracy_score(Y_test, preds_lasso)}")
print(f"Precision: {metrics.precision_score(Y_test, preds_lasso)}")
print(f"Recall: {metrics.recall_score(Y_test, preds_lasso)}")
print(f"F1: {metrics.f1_score(Y_test, preds_lasso)}")
print(metrics.classification_report(Y_test, preds_lasso))

Fitting 10 folds for each of 23 candidates, totalling 230 fits
[CV 1/10] END ...............C=0.01, penalty=l1;, score=0.800 total time=   1.4s
[CV 2/10] END ...............C=0.01, penalty=l1;, score=0.800 total time=   1.2s
[CV 3/10] END ...............C=0.01, penalty=l1;, score=0.800 total time=   1.2s
[CV 4/10] END ...............C=0.01, penalty=l1;, score=0.800 total time=   1.2s
[CV 5/10] END ...............C=0.01, penalty=l1;, score=0.800 total time=   1.2s
[CV 6/10] END ...............C=0.01, penalty=l1;, score=0.833 total time=   1.2s
[CV 7/10] END ...............C=0.01, penalty=l1;, score=0.833 total time=   1.1s
[CV 8/10] END ...............C=0.01, penalty=l1;, score=0.833 total time=   1.4s
[CV 9/10] END ...............C=0.01, penalty=l1;, score=0.833 total time=   1.2s
[CV 10/10] END ..............C=0.01, penalty=l1;, score=0.792 total time=   1.3s
[CV 1/10] END ...............C=0.02, penalty=l1;, score=0.800 total time=   0.2s
[CV 2/10] END ...............C=0.02, penalty=l

Same exact performance. Let's see the parameters and coefficients:

In [103]:
print(tuned_lasso.best_estimator_)

LogisticRegression(C=10, max_iter=10000, penalty='l1', solver='saga')


In [106]:
lasso = LogisticRegression(C=10, penalty='l1', solver='saga', max_iter=10000)

lasso.fit(X_train, Y_train)
print(lasso.coef_)

[[ 8.23777732e-02  3.05587201e-02  0.00000000e+00 -1.72447569e-01
   0.00000000e+00  0.00000000e+00 -7.91405523e-02  9.60643537e-02
  -2.71926144e-01 -1.94531458e-01  6.30643741e-02 -1.21128039e-02
  -1.97721356e-02  8.15968201e-02 -3.73268650e-02 -1.52745566e-01
   0.00000000e+00  0.00000000e+00  1.38569692e-01  4.53118512e-02
   1.86479283e-02 -4.99885563e-01 -1.34846477e-01 -2.43547244e-01
  -3.54650796e-03 -9.67660880e-02  0.00000000e+00  8.30517198e-02
  -4.01163265e-01  1.26066384e-01 -1.43210772e-01 -4.33165908e-02
   0.00000000e+00 -1.96567077e-02  0.00000000e+00 -2.02756112e-02
  -9.56723520e-02 -2.53594572e-01  3.58033461e-01 -2.03342033e-01
  -2.56081717e-01 -1.29955560e-01 -5.66264936e-02  1.14276381e-01
   2.38323381e-01  1.96232623e-01  2.26529254e-01  1.93804310e-01
   0.00000000e+00  1.68833917e-08 -2.37824878e-01  2.08829768e-01
   7.00656149e-02  2.11981548e-01  4.80930576e-01  6.04857377e-01
   9.62707297e-02  4.42968970e-02 -3.79462271e-04  6.43523833e-03
  -1.41990

The coefficients are different, but the performance remains the same and the ridge is also quicker to fit.